In [2]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions.{col, desc, hour, regexp_extract, split, sum, to_date}

import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions.{col, desc, hour, regexp_extract, split, sum, to_date}


In [3]:
val sparkSession = SparkSession
    .builder()
    .master("local[*]")
    .appName("curso")
    .config("spark.some.config.option", "some-value")
    .getOrCreate()

sparkSession: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@398ac9eb


In [5]:
var leer = sparkSession.read
    .option("header", "true")
    .option("sep", " ")
    .option("inferSchema", "true")
    .csv("nasa_aug95.csv")

leer: org.apache.spark.sql.DataFrame = [requesting_host: string, datetime: timestamp ... 3 more fields]


In [6]:
leer.show(false)

+---------------------------+-------------------+----------------------------------------------------------------+------+-------------+
|requesting_host            |datetime           |request                                                         |status|response_size|
+---------------------------+-------------------+----------------------------------------------------------------+------+-------------+
|in24.inetnebr.com          |1995-08-01 00:00:01|GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt HTTP/1.0    |200   |1839         |
|uplherc.upl.com            |1995-08-01 00:00:07|GET / HTTP/1.0                                                  |304   |0            |
|uplherc.upl.com            |1995-08-01 00:00:08|GET /images/ksclogo-medium.gif HTTP/1.0                         |304   |0            |
|uplherc.upl.com            |1995-08-01 00:00:08|GET /images/MOSAIC-logosmall.gif HTTP/1.0                       |304   |0            |
|uplherc.upl.com            |1995-08-01 00:00:08

In [7]:
leer.printSchema()

root
 |-- requesting_host: string (nullable = true)
 |-- datetime: timestamp (nullable = true)
 |-- request: string (nullable = true)
 |-- status: integer (nullable = true)
 |-- response_size: integer (nullable = true)



In [8]:
val prot = leer.withColumn("new_row",split(col("request"), " "))
    .select(col("new_row").getItem(2).as("Protocolos"))
    .filter(col("Protocolos").like("%HT%/%")).distinct().show()

+-------------+
|   Protocolos|
+-------------+
|    HTTP/V1.0|
|     HTTP/1.0|
|     HTML/1.0|
|HTTP/1.0From:|
+-------------+



prot: Unit = ()


In [9]:
val stat = leer.groupBy("Status").count()
    .orderBy(desc("Count")).show()

+------+-------+
|Status|  count|
+------+-------+
|   200|1398988|
|   304| 134146|
|   302|  26497|
|   404|  10054|
|   403|    171|
|   501|     27|
|   500|      3|
|   400|      2|
+------+-------+



stat: Unit = ()


In [10]:
val pet = leer.withColumn("new_row",split(col("request"), " "))
    .select(col("new_row").getItem(0).as("Peticion"))
    .groupBy("Peticion").count().orderBy(col("count").desc).show()

+-----------------+-------+
|         Peticion|  count|
+-----------------+-------+
|              GET|1565810|
|             HEAD|   3965|
|             POST|    111|
|huttle/countdown/|      1|
|                ?|      1|
+-----------------+-------+



pet: Unit = ()


In [11]:
val tran = leer.filter(col("response_size").isNotNull)
    .select(col("requesting_host"), col("response_size"))
    .orderBy(col("response_size").desc).limit(1).show()

+---------------+-------------+
|requesting_host|response_size|
+---------------+-------------+
| 163.205.156.16|      3421948|
+---------------+-------------+



tran: Unit = ()


In [12]:
val tra = leer.groupBy(to_date(col("datetime"))).count()
    .orderBy(col("count").desc).limit(1).show()

+-----------------+-----+
|to_date(datetime)|count|
+-----------------+-----+
|       1995-08-31|90122|
+-----------------+-----+



tra: Unit = ()


In [13]:
val hor = leer.groupBy(hour(col("datetime"))).count()
    .orderBy(col("count")).show()

+--------------+-----+
|hour(datetime)|count|
+--------------+-----+
|             4|26756|
|             5|27587|
|             3|29995|
|             6|31287|
|             2|32508|
|             1|38531|
|             7|47386|
|             0|47862|
|            23|54570|
|            21|57985|
|            19|59315|
|            20|59944|
|            22|60673|
|             8|65443|
|            18|66809|
|             9|78695|
|            17|80834|
|            10|88309|
|            11|95340|
|            16|99527|
+--------------+-----+
only showing top 20 rows



hor: Unit = ()


In [14]:
val err = leer.filter(col("status").equalTo(404))
    .groupBy(to_date(col("datetime"))).count()
    .orderBy(col("count")).show()

+-----------------+-----+
|to_date(datetime)|count|
+-----------------+-----+
|       1995-08-12|  196|
|       1995-08-19|  209|
|       1995-08-13|  216|
|       1995-08-05|  236|
|       1995-08-01|  243|
|       1995-08-18|  255|
|       1995-08-16|  259|
|       1995-08-11|  263|
|       1995-08-17|  271|
|       1995-08-09|  279|
|       1995-08-14|  287|
|       1995-08-22|  288|
|       1995-08-03|  303|
|       1995-08-21|  305|
|       1995-08-20|  312|
|       1995-08-10|  315|
|       1995-08-15|  327|
|       1995-08-23|  345|
|       1995-08-04|  346|
|       1995-08-26|  366|
+-----------------+-----+
only showing top 20 rows



err: Unit = ()
